---
title: "Using JavaScript and D3 in Jupyter with Deno"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["D3", "Visualization", "JavaScript", "TypeScript"]
format:
  html:
    code-fold: false
toc-depth: 4
jupyter: python3
highlight-style: github
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="module" src="./javascript/zoom-in.js"></script>
    <script type="module" src="./javascript/show-tooltip.js"></script>
---

# Introduction

In this post we will set up [D3](https://d3js.org/) and JavaScript inside a Jupyter notebook using [Deno](https://deno.com/). Deno is "the open-source JavaScript runtime for the modern web." Importantly for us, it will enable the use modern JavaScript, TypeScript, and `npm` inside Jupyter notes.

## Jupyter-Deno Setup

In order to run JavaScript and, by extension, D3 code inside Jupyter notebooks we need a JavaScript runtime environment (a JS kernel for the Jupyter notebook). [Deno](https://deno.com/) is a modern contender to [Node](https://nodejs.org/en). Download and install Deno by issuing the following cURL.

```
curl -fsSL https://deno.land/install.sh | sh
```

We now need to register Deno as a kernel for Jupyter. The following command does the trick.

```
deno jupyter --install
```

We also need to activate the Deno kernel. To do so, just issue the following command.

```
deno jupyter --unstable
```

Finally, inside our Jupyer notebook (or inside VS Code using the Jupyter plugin), we need to select the Deno kernel.

After all of this, let's verify that we can run JavaScript inside *this* Juptyer notebook.

In [1]:
console.log("Hello from this Jupyter notebook!")

Hello from this Jupyter notebook!


## Example Visualization - An Interactive Map of the United States

We will use [GeoJSON](https://geojson.org/), and its extension [TopoJSON](https://github.com/topojson/topojson), as our map data in order to visualize an interactive map of the United States with county-level granularity. The TopoJSON project is more than just a raw re-representation of the GeoJSON US political map. It also provides an interface to manipulate the raw data in various useful ways. We will import the raw data as well as the extra tooling surrounding it. But first, let's focus on getting the raw data.

### Downloading Geographic Data

We use this convenient [re-distribution](https://github.com/topojson/us-atlas/tree/master) of the US map data.

In [18]:
// Fetch the US map TopoJSON data
const url = "https://cdn.jsdelivr.net/npm/us-atlas@3/states-10m.json"; 
const us = await (await fetch(url)).json();

### Examining the Downloaded Geographic Data

If we examine the TopoJSON file `us` obtained above, we can notice useful patterns. For example, we can log the first state's `id`, and `name`.

In [19]:
console.log(JSON.stringify(us.objects.states.geometries.slice(0,1), ["id", "properties", "name"], 2))

[
  {
    "id": "01",
    "properties": {
      "name": "Alabama"
    }
  }
]


Now let's do the same for the counties TopoJSON dataset. 

In [20]:
// Fetch the US map TopoJSON data
const url = "https://cdn.jsdelivr.net/npm/us-atlas@3/counties-10m.json"; 
const us = await (await fetch(url)).json();

In [21]:
console.log(JSON.stringify(us.objects.counties.geometries.slice(0,1), ["id", "properties", "name"], 2))

[
  {
    "id": "04015",
    "properties": {
      "name": "Mohave"
    }
  }
]


Notice that both states and counties are represented in a similar JSON format (specifically, TopoJSON). They both have an `id` field (which uniquely identifies a region), and a `properties.name` field containing the name of the region.

As it turns out, there is further structure in this representation that we can exploit. We can easily identify all counties belonging to a given state. The field `counties.geometry.id` is tied to `states.geometries.id` by its first two digits. Meaning, if `Mohave` county above has `id = 04015`, then the state of `Arizona` must have `id = 04`. 

Let's define a utility function to do just that.

In [22]:
// Helper function to get the counties of a given state based on `state.id`
const getCountiesOfState = (us, stateId) => 
    us.objects.counties.filter((county) => county.id.slice(0,2) == stateId);

### Choosing the Right Tools

#### D3 - A Versatile Visualization Library

First order of business is importing [D3](https://d3js.org/). With Deno, it is as simple as:

In [23]:
import * as d3 from "npm:d3"

D3 is a JavaScript library for visualization. The three D-s in "D3" stand for **"Data Driven Documents."** Think DOM elements that are styled, positioned, scaled, etc. according to some attached source of data.

In fact, a core pattern in D3 is the following:

1. Select some array of DOM elements
2. "Attach" an array of data to that array of DOM elements
3. Perform some actions for each DOM element formed from the data array

The first two steps above roughly translates to the following API.

In [ ]:
document.selectAll("div")
  .data(correspondingDataset)
  .join("div")

We can imagine `<divs>` being bars in a bar graph that are scaled vertically according to the values inside the `correspondingDataset`.

The third step, that of performing further actions on the D3 selection, follows the same pattern of chained function calls. At each step, a D3 `Selection` is returned and ready to be manipulated again. Some function calls return an entirely new `Selection`, others the same `Selection` but with the attributes of the elements tweaked (for example). In D3, by convention, calls that return a new `Selection` are indented by 2 spaces, and those that return the same `Selection` are indented by 4. This gives us a visual clue as to what's the latest `Selection` we're trying to manipulate with D3.

#### Other Tools: TopoJSON Tooling, Linkedom, etc.

We will also need the *means* to work with the downloaded data, and to ultimately display the data as DOM elements (counties, states, etc.) using D3 inside a browser-less JavaScript runtime. But why do we need special tools for this? Can't we just write the D3 code? Well, it's not that simple. These notes are written on a browser-less Deno runtime environment, It's Chrome (or browsers in general) that implements an [HTML DOM API](https://developer.mozilla.org/en-US/docs/Web/API/HTML_DOM_API), and/or an [HTML Canvas API](https://www.w3schools.com/Jsref/api_canvas.asp) on top of the V8 engine that runs JavaScript in the browser. So, we're going to need a way to create DOM elements using just the Deno runtime, without relying on Chrome.

For DOM support, and to render rich media types (like HTML, Markdown, SVG, etc.) inside a Deno environment, we have two options. The first is to use the [skia-canvas](https://github.com/samizdatco/skia-canvas) module, the second is to use a Deno (or Node.js) mock DOM library like [Linkedom](https://github.com/WebReflection/linkedom) which offers a fake DOM API that goes on top of Deno's runtime.

In addition, we will need the utilities from TopoJSON to work with that data format. We need [feature](https://github.com/topojson/topojson-client/blob/master/README.md#feature) (to convert TopoJSON to a representation closer to GeoJSON which is expected by D3's `geoPath`) and [mesh](https://github.com/topojson/topojson-client/blob/master/README.md#mesh) (for, possibly, extracting borders).

From D3 we'll specifically need [geoPath](https://d3js.org/d3-geo/path#geoPath) -- a geographic path generator with default parameters such as the type of geographic projection into 2D. And, even though there *is* a default projection for `geoPath`, we can also import the `d3.geoAlbersUsa` projection explicitly.

### Rendering the Map

As a minimal example, let's focus on just displaying the map from the data we downloaded and converted earlier. We will add interactive features later.

The following code uses Linkedom to create a new HTML document, fetches the TopoJSON data, uses the D3 pattern mentioned above to bind the data to the DOM elements, and, finally, uses D3 to draw the states as `<paths>` inside an `<svg>` wrapper of the map.

Note that we need to, first, convert the TopoJSON into a format similar to GeoJSON using the `topojson.feature` method. We, then, project the resulting `Feature`-s, before supplying them to the method `d3.geoPath` which generates the path data (attribute `d` of HTML `<path>` elements).

The following is all the code that's required to render the static map of US states. It's also only a matter of adding a couple of more lines to render the full map, including the counties (see the last section for an interactive county-level map).

In [25]:
// Imports: D3, TopoJSON, and Linkedom
import * as d3 from "npm:d3";
import * as topojson from "npm:topojson";
import { DOMParser } from "npm:linkedom";

// Create a fake DOM via Linkedom
const document = new DOMParser().parseFromString(
  `<!DOCTYPE html><html lang="en"><head></head><body></body></html>`,
  "text/html",
);

// Fetch and parse the US map TopoJSON
const url = "https://cdn.jsdelivr.net/npm/us-atlas@3/states-10m.json";
const us = await (await fetch(url)).json();
const states = topojson.feature(us, us.objects.states) as any; // GeoJSON 'FeatureCollection'

// Create an <svg> wrapper in Linkedom's fake DOM using D3
const width = 975;
const height = 610;
const body = d3.select(document.body);
const svg = body
  .append("svg")
    .attr("xmlns", "http://www.w3.org/2000/svg") 
    .attr("width", width)
    .attr("height", height)
    .attr("viewBox", `0 0 ${width} ${height}`);
const group = svg.append("g");
const groupOfStates = group.append("g")
    .attr("fill", "#EBE8E7")
    .attr("stroke", "#191516")

// Create a geo-projection & a path generator
const projection = d3.geoAlbersUsa().fitSize([width, height], states);
const path = d3.geoPath(projection);

// Append one <path> per state
groupOfStates.selectAll("path")
  .data(states.features)
  .join("path")
    // Define the 'd' attribute of the <path> which gives it its shape
    .attr("d", path);
  

// Get the markup from Linkedom to display it
const htmlOutput = document.documentElement.outerHTML; // Entire <html>…</html>

// Use 'Deno.jupyter.display' shorthand to display the raw <html> string as actual HTML
await Deno.jupyter.display({
  "text/html": htmlOutput,
}, { raw: true });

<path d="M677.986,480.51L681.705,479.622L681.734,479.149L682.737,479.819L682.176,480.392L681.478,480.149L680.27,480.286L678.951,480.714ZM671.856,376.314L671.833,376.032L681.341,375.336L685.329,374.981L695.477,374.28L701.363,373.74L703.7,373.467L713.76,372.492L717.625,372.167L719.238,378.089L721.621,386.24L723.563,393.221L725.015,398.64L726.636,404.36L730.288,417.473L730.94,418.005L730.758,418.709L731.699,419.454L732.345,422.206L733.151,423.478L734.408,424.641L734.782,426.152L735.448,426.757L735.05,429.021L736.287,429.408L737.159,430.064L736.619,431.113L735.899,431.197L736.083,431.63L735.212,432.13L734.474,433.201L734.791,434.225L734.846,435.848L734.573,437.509L733.808,438.96L733.707,439.824L734.368,442.096L734.331,442.934L735.77,444.415L736.228,446.634L735.889,447.752L736.114,448.54L735.757,450.095L735.993,451.562L735.575,452.008L736.016,452.922L735.933,453.953L737.456,455.498L738.059,456.639L738.324,457.762L733.873,458.274L729.165,458.897L721.86,459.767L710.286,460.964L701.288,461.71L690.237,462.762L690.5,463.455L689.823,465.683L689.995,466.102L691.662,467.443L691.93,468.192L694.471,469.472L694.789,470.744L693.957,472.741L694.251,473.751L695.679,474.583L694.531,475.346L694.132,476.927L693.553,477.262L694.063,477.668L693.099,478.244L690.603,479.125L687.953,479.808L684.192,480.205L685.338,479.336L686.186,479.753L688.496,479.045L688.643,478.393L687.557,477.583L687.018,476.819L685.6,476.056L684.987,474.413L685.472,472.978L685.15,471.573L684.605,470.716L683.216,470.23L682.285,471.175L682.478,471.875L681.964,472.991L682.095,473.753L681.743,474.385L681.9,477.083L681.465,478.362L680.463,478.52L680.475,477.841L679.071,477.204L678.087,477.585L677.717,477.182L676.541,477.691L676.007,473.554L675.272,466.888L674.239,458.998L672.425,444.56L672.595,439.204L672.903,425.766L672.908,421.758L673.33,400.534L673.468,397.494L673.684,385.297L673.881,378.421L672.812,377.846Z" /><path d="M162.053,531.019L162.118,530.765L162.358,530.823L162.112,531.121ZM160.778,544.929L161.151,544.318L161.365,544.419L161.366,544.722L161.002,545.228L160.787,545.113ZM157.91,546.468L158.277,546.546L158.244,546.784ZM157.173,546.613L157.485,546.714L157.394,546.934L157.195,546.849ZM156.605,546.375L156.885,546.379L156.823,546.605ZM156.723,553.623L156.829,553.323L157.119,553.4L157.02,553.883L156.807,554.002ZM155.834,548.248L155.997,548.059L156.301,548.313L155.836,548.34ZM155.477,537.365L155.853,536.834L155.938,536.094L156.36,536.271L155.997,536.704L155.886,537.03L155.958,537.186ZM154.723,548.35L154.911,547.983L155.169,548.161L155.494,548.178L155.035,548.408ZM153.414,538.804L153.644,538.76L153.724,539.338ZM150.61,554.937L151.022,554.348L151.238,554.306L151.618,554.736L151.689,554.603L151.381,554.18L151.463,553.824L151.592,553.756L151.982,553.915L152.282,553.786L151.861,553.437L152.085,553.066L152.504,553.263L152.719,553.233L152.512,552.868L152.682,552.754L152.971,552.88L153.166,552.581L152.908,552.546L152.718,552.332L153.004,552.308L153.31,551.861L153.793,551.733L153.617,551.507L153.565,551.06L153.94,550.618L154.044,550.82L154.79,550.467L154.89,550.511L154.739,551.291L154.556,551.413L154.201,552.099L154.285,552.65L154.799,552.565L154.816,552.111L155.132,552.162L155.445,552.621L155.722,552.298L155.9,552.51L155.678,553.286L155.812,553.427L155.927,552.95L156.305,552.629L156.41,552.791L156.39,553.548L155.976,554.146L155.511,554.007L155.339,553.479L155.041,553.658L155.245,554.304L154.932,554.443L154.367,554.371L154.145,554.732L154.024,554.497L154.042,554.01L153.898,554L153.784,554.752L153.672,554.912L153.226,555.033L153.26,555.276L153.031,555.424L152.381,555.521L151.131,555.037L150.828,555.126ZM149.979,544.78L150.547,544.591L150.8,544.872L150.733,545.188L150.553,545.295L150.15,545.318L149.924,544.931ZM147.609,565.441L148.075,565.147L148.195,565.462L147.639,565.52ZM146.747,566.581L147.051,566.175L147.337,566.045L147.502,566.118L147.879,566.007L147.954,566.27L148.24,566.513L148.782,566.407L148.752,566.65L148.466,566.854L147.772,566.893L147.289,566.977L146.806,566.811ZM145.073,560.266

To render the resulting DOM we use `Deno.jupyer.display` which enables us to render objects as *rich media types* (such as HTML, Markdown, SVG, etc.). This builds upon Jupyter's support of [mimetypes](https://docs.jupyter.org/en/latest/reference/mimetype.html). All we have to do, to display the above DOM, is to simply create a wrapper object that invokes the function `Jupyter.display`. In fact, the last few lines of the code above use a shorthand for exactly doing that. HHowever, here's a minimal example of using `Deno.jupyter.display` (without the shorthand) to render an HTML paragraph for educational purposes.

In [26]:
{
    [Symbol.for('Jupyter.display')]: () => ({
      'text/html': '<p> Hello from this Jupyter notebook!<p>'
    })
}

Hello from this Jupyter notebook!

### Adding Interactivity

We saw how to render the map as non-inteactive `<svg>` element, but it's pretty useless by itself without interactivity features. Let's see how to add those!

In D3, we may register a JavaScript event listener in one of two ways: 

* By using D3’s [selection.on](https://d3js.org/d3-selection/events#selection_on) API 
* By using [selection.attr](https://d3js.org/d3-selection/modifying#selection_attr) API (for the most basic of interactivity features) 

In this section, we will use the attribute-based API to change the color as soon as a user hovers over a state. We will then use the `selection.on` API to add more complex interactions (like tooltips, zoom, and pan). Both attribute-based event listeners and those event listeners which get added by `selection.on` work in tandem which can, potentially, introduce styling conflicts. Nevertheless, we explore both options below. 


#### Basic Interactivity - Hover Effects, Tooltips

Event listeners added by using the `selection.attr` API directly alter the HTML attributes (e.g. `<path onmouseover="...">...</path>`). 

Depending on the framework we use, the DOM may be re-rendered multiple times during the framework's DOM lifecycle, however attribute-based event listeners will *always* be there after each re-render. Because of this, we can safely add them inside our Deno-Jupyter notebook cells without fear of them getting lost in the DOM lifecycle. As we'll see in the next section that's not necessarily the case when using D3's `selection.on` API. *That* will require the use of a workaround. But, for now, we're in the happy world of attribute-based DOM events.

We add an attribute-based hover effect to change the color of a state on a `mouseover` event, like so:

```js
groupOfStates.selectAll("path")
  .data(states.features)
  .join("path")
    // Basic Interactivity: Hover to change color
    .attr("cursor", "pointer")
    .attr("onmouseover", "this.style.fill = '#F9C22E';")
    .attr("onmouseout", "this.style.fill = '#EBE8E7';")
```

Most of the code to generate the interactive map is the same, but feel free to expand the code block below to examine the critical differences.

<details><summary>Click to expand interactive map code</summary>

In [ ]:
// Imports: D3, TopoJSON, and Linkedom
import * as d3 from "npm:d3";
import * as topojson from "npm:topojson";
import { DOMParser } from "npm:linkedom";

// Create a fake DOM via Linkedom
const document = new DOMParser().parseFromString(
  `<!DOCTYPE html><html lang="en"><head></head><body></body></html>`,
  "text/html",
);

// Fetch and parse the US map TopoJSON
const url = "https://cdn.jsdelivr.net/npm/us-atlas@3/states-10m.json";
const us = await (await fetch(url)).json();
const states = topojson.feature(us, us.objects.states) as any; // GeoJSON 'FeatureCollection'

// Create an <svg> wrapper in Linkedom's fake DOM using D3
const width = 975;
const height = 610;
const body = d3.select(document.body);
body
  .append("p")
  .text("💡 Tip: Try hovering over a state!")
  .style("text-align", "center")
const svg = body
  .append("svg")
    // Add an `id` (the external hook responsible for displaying the tooltip attaches a tooltip element to this wrapper by `id`)
    .attr("id", "state-map-wrapper")
    .attr("width", width)
    .attr("height", height)
    .attr("viewBox", `0 0 ${width} ${height}`);
const group = svg.append("g");
const groupOfStates = group.append("g")
    // Add an `id` (the external hook responsible for zoom behavior will select this element by `id`)
    .attr("id", "group-of-states")
    .attr("fill", "#EBE8E7")
    .attr("stroke", "#191516")

// Create a geo-projection & a path generator
const projection = d3.geoAlbersUsa().fitSize([width, height], states);
const path = d3.geoPath(projection);


// Append one <path> per state
groupOfStates.selectAll("path")
  .data(states.features)
  .join("path")
    // Define the 'd' attribute of the <path> which gives it its shape
    .attr("d", path)
    // Basic Interactivity: Hover to change color
    .attr("cursor", "pointer")
    .attr("onmouseover", "this.style.fill = '#F9C22E';")
    .attr("onmouseout", "this.style.fill = '#EBE8E7';")
    // Add an `id` (needed for selecting these elements using D3 in the external script that's responsible for displaying the tooltip)
    .attr("id", "state-path")
    // Add a custom attribute `state-name` (needed for passing along the state name to the external script)
    .attr("state-name", (d) => {return d.properties.name})
    // Add custom attribute for the centroid of each state <path> (needed to position the tooltip correctly in the external script)
    .attr("cx", (d) => {
      return path.centroid(d)[0]; // Index 0 corresponds to the x-coordinate of the centroid
    })
    .attr("cy", (d) => {
      return path.centroid(d)[1]; // Index 1 corresponds to the y-coordinate of the centroid
    })
    // Custom attributes for the bounding box of each state, needed for the external zoom script 
    .attr("x0", (d) => {
      return path.bounds(d)[0][0]; // Index 0,0 corresponds to the x0 of the bounding box
    })
    .attr("y0", (d) => {
      return path.bounds(d)[0][1];
    })
    .attr("x1", (d) => {
      return path.bounds(d)[1][0];
    })
    .attr("y1", (d) => {
      return path.bounds(d)[1][1];
    })
    ;

</details>

##### Tooltips

We add the following D3 script, `show-tooltip.js`, within this post's sub-directory. But 2hy we break up this code into its own external script? That will be explained in the next section of **Advanced Interactivity** (*hint: we need the script to run at the end of the DOM lifecycle, after the DOM's been rendered the final time*). 

In D3, we can attach an event listener using the [selection.on](https://d3js.org/d3-selection/events#selection_on) method. The syntax is as follows:

```js
selection.on("mouseover", function(event, d) {
  // Our custom behavior here
});
```
Where `mouseover` can be any event (e.g. `click`). 

The callback function receives specific parameters `event` and `d` (or `data`) that provide context about the event, and the data associated with the selected elements. 

The `event` object has useful fields like the mouse position, which keys were pressed (along which modified keys), etc. Here are some useful `event` fields:

| `event` Field | Explanation |
|----------|----------|
| `event.target`    | The DOM element that was clicked, scrolled over, or otherwise interacted with |
| `event.currentTarget`    | The DOM element to which the event listener is attached (not the same as `target`), otherwise also accessible as `self` within the callback function  |
| `event.transform`    | The type (e.g. zoom/rotation/etc.) and exact specifications of the event (e.g. how much we've panned, zoomed, etc.)   |
| `event.scale`    | `event.transform.k` -- The magnitude of the zoom |
| `event.translate`    | `event.transform.x` and `y` -- The translation vector of the event   |
| `event.matrix`| The transformation matrix of the event (in case it's not a simple zoom, pan, etc.) |
| `event.rotate`| The rotation component of the event |
| `event.skewX` or `skewY` | Horizontal and vertical skew |

Of course, not all events will have all of these fields. A `click` event, for example, won't have a rotational component...

In the tooltip script below we use `event.target.getAttribute("cx")` and `event.target.getAttribute("cy")` to get the coordinates of the centroid of the state `<path>` attached to it as an attribute.

The `data` object contains all the data bound by D3 to the DOM element. As such, it'll have fields defined by us (or by some dataset we're using).

Here's the external script for the tooltip, feel free to expand and examine!

<details><summary>Click to expand tooltip code</summary>

```js
import * as d3 from "https://cdn.jsdelivr.net/npm/d3@7/+esm";

/*---------------------------------------*/
////////////////// MAP 1 //////////////////
/*---------------------------------------*/

// Selects all elements with `id=state-path`and attaches an event-listener for a `mouseover` event
d3.selectAll("#state-path")
        .on("mouseover", showTooltips)
        .on("mouseout", hideTooltips);

// Tooltip
const svg = d3.select("#state-map-wrapper").node(); // Grab the DOM node, the DOM API provides `parentNode`, a method to getb the parent of an element
const parentNode = svg.parentNode;
const parentSelection = d3.select(parentNode);
const tooltip = parentSelection
    .append("div")
        .attr("id", "state-name-tooltip")
        .style("pointer-events", "none") // So that the tooltip doesn’t block mouse events
        .style("opacity", "0") // Hidden by default...
        .style("position", "relative")
        .style("width", "max-content")
        .style("padding", "4px 8px")
        .style("border", "1px solid #191516")
        .style("top", "0px")
        .style("left", "0px")
        .style("background", "#EBE8E7")
        .style("color", "#191516");

/*------------------------------------------*/
////////////////// Behavior //////////////////
/*------------------------------------------*/

// Get global state state exclusively in zoom-in.js
let activated = d3.select("#zoomable-state-path").attr("activated") == "true" ? true : false; // Checks the first element only

function showTooltips(event, d) {
    const left = parseInt(event.target.getAttribute("cx"));
    const top = parseInt(event.target.getAttribute("cy")) - 610;  // The additive term is the height of the <svg> wrapper

    activated = event.target.getAttribute("activated") == "true" ? true : false; // Whether a state has been clicked into or not

    // Use for debugging
    // console.log(activated);

    tooltip
            // Set the state name displayed inside the tooltip
            .text(`${event.target.getAttribute("state-name")}`)
            .style("opacity", "1")
            // Set the position of the tooltip
            .style("left", `${left}px`)
            .style("top", `${top}px`);
}

function hideTooltips(event, d) {
    tooltip
            .style("opacity", "0");
}
```

</details>

Let's re-render the map.

In [36]:
// Get the new markup from Linkedom to display it
const htmlOutput = document.documentElement.outerHTML; // Entire <html>…</html>

// Use 'Deno.jupyter.display' to display the raw <html> string, now containing the county <paths>-s, as actual HTML
await Deno.jupyter.display({
  "text/html": htmlOutput,
}, { raw: true });

💡 Tip: Try hovering over a state! <path y1="480.71434197353415" x1="738.3236739367742" y0="372.1672157529463" x0="671.8325500230226" cy="422.637486026742" cx="700.7386861292639" state-name="Alabama" id="state-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M677.986,480.51L681.705,479.622L681.734,479.149L682.737,479.819L682.176,480.392L681.478,480.149L680.27,480.286L678.951,480.714ZM671.856,376.314L671.833,376.032L681.341,375.336L685.329,374.981L695.477,374.28L701.363,373.74L703.7,373.467L713.76,372.492L717.625,372.167L719.238,378.089L721.621,386.24L723.563,393.221L725.015,398.64L726.636,404.36L730.288,417.473L730.94,418.005L730.758,418.709L731.699,419.454L732.345,422.206L733.151,423.478L734.408,424.641L734.782,426.152L735.448,426.757L735.05,429.021L736.287,429.408L737.159,430.064L736.619,431.113L735.899,431.197L736.083,431.63L735.212,432.13L734.474,433.201L734.791,434.225L734.846,435.848L734.573,437.509L733.808,438.96L733.707,439.824L734.368,442.096L734.331,442.934L735.77,444.415L736.228,446.634L735.889,447.752L736.114,448.54L735.757,450.095L735.993,451.562L735.575,452.008L736.016,452.922L735.933,453.953L737.456,455.498L738.059,456.639L738.324,457.762L733.873,458.274L729.165,458.897L721.86,459.767L710.286,460.964L701.288,461.71L690.237,462.762L690.5,463.455L689.823,465.683L689.995,466.102L691.662,467.443L691.93,468.192L694.471,469.472L694.789,470.744L693.957,472.741L694.251,473.751L695.679,474.583L694.531,475.346L694.132,476.927L693.553,477.262L694.063,477.668L693.099,478.244L690.603,479.125L687.953,479.808L684.192,480.205L685.338,479.336L686.186,479.753L688.496,479.045L688.643,478.393L687.557,477.583L687.018,476.819L685.6,476.056L684.987,474.413L685.472,472.978L685.15,471.573L684.605,470.716L683.216,470.23L682.285,471.175L682.478,471.875L681.964,472.991L682.095,473.753L681.743,474.385L681.9,477.083L681.465,478.362L680.463,478.52L680.475,477.841L679.071,477.204L678.087,477.585L677.717,477.182L676.541,477.691L676.007,473.554L675.272,466.888L674.239,458.998L672.425,444.56L672.595,439.204L672.903,425.766L672.908,421.758L673.33,400.534L673.468,397.494L673.684,385.297L673.881,378.421L672.812,377.846Z" /><path y1="587.8605689458193" x1="250.35186397980095" y0="453.74685748545426" x0="-1.1368683772161603e-13" cy="510.1736115713112" cx="153.70158104833692" state-name="Alaska" id="state-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M162.053,531.019L162.118,530.765L162.358,530.823L162.112,531.121ZM160.778,544.929L161.151,544.318L161.365,544.419L161.366,544.722L161.002,545.228L160.787,545.113ZM157.91,546.468L158.277,546.546L158.244,546.784ZM157.173,546.613L157.485,546.714L157.394,546.934L157.195,546.849ZM156.605,546.375L156.885,546.379L156.823,546.605ZM156.723,553.623L156.829,553.323L157.119,553.4L157.02,553.883L156.807,554.002ZM155.834,548.248L155.997,548.059L156.301,548.313L155.836,548.34ZM155.477,537.365L155.853,536.834L155.938,536.094L156.36,536.271L155.997,536.704L155.886,537.03L155.958,537.186ZM154.723,548.35L154.911,547.983L155.169,548.161L155.494,548.178L155.035,548.408ZM153.414,538.804L153.644,538.76L153.724,539.338ZM150.61,554.937L151.022,554.348L151.238,554.306L151.618,554.736L151.689,554.603L151.381,554.18L151.463,553.824L151.592,553.756L151.982,553.915L152.282,553.786L151.861,553.437L152.085,553.066L152.504,553.263L152.719,553.233L152.512,552.868L152.682,552.754L152.971,552.88L153.166,552.581L152.908,552.546L152.718,552.332L153.004,552.308L153.31,551.861L153.793,551.733L153.617,551.507L153.565,551.06L153.94,550.618L154.044,550.82L154.79,550.467L154.89,550.511L154.739,551.291L154.556,551.413L154.201,552.099L154.285,552.65L154.799,552.565L154.816,552.111L155.132,552.162L155.445,552.621L155.722,552.298L155.9,552.51L155.678,553.286L155.812,553.427L155.927,552.95L156.305,552.629L156.41,552.791L156.39,553.548L155.976,554.146L155.511,554.007L155.339,553.479L155.041,553.658L155.245,554.304L154.932,554.443L1

#### Advanced Interactivity - Drag and Zoom, Click to Zoom

Event listeners using D3’s own [selection.on](https://d3js.org/d3-selection/events#selection_on) do not directly change the HTML attributes (e.g. `onmouseover="..."`) of the `<path>` elements in the DOM (unlike when using the [selection.attr](https://d3js.org/d3-selection/modifying#selection_attr) API). Instead, `selection.on` attaches an event listener in-memory -— conceptually it's the same as calling `element.addEventListener("mouseover", callback)` in vanilla JavaScript. 

Event listeners added with `selection.on` may fail to trigger in the browser (depending on the framework we're using to generate the final DOM). Often, a custom renderer (of some sort) in a framework we've adopted can *re-draw* the DOM after we've attached listeners to the elements using D3's `selection.on` API. This was the case when adding the tooltips to the states in the interactive map above. The event listeners were being successfully attached in the Deno-Jupyter environment this post is written in, initially, but the *same* event listeners were missing in the final DOM that was rendered within the browser. 

::: {.callout-tip title="💡 TIP: Debugging Events in the Browser" appearance="minimal" collapse="true"}

D3 has a small convenience. If we do:

```js
const currentListener = d3.select(someElement).on("click");
```

And:

* An event listener is currently assigned to the [HTML DOM event](https://www.w3schools.com/jsref/dom_obj_event.asp) known as `"click"` on the *first* element that matches the query in the D3 selection, then `currentListener` will be the callback function assigned to that event 
* Else, it will be `undefined`. 

This gives us a good way to debug event listener attachment.

If the events aren't triggering, for some reason, make sure to check the value of `currentListener` inside the browser's console (don't just log it inside the Deno-Jupyter notebook). The DOM may be getting re-rendered, at some point, replacing a `<path>` element *after* D3 attached the event listener with `selection.on`. This may cause the attached listener to be lost. The source of truth is always the browser's environment, not the Deno-Jupyter environment. 

Note that we'll have to import D3 within the browser's console because, by default, it won't be inside the global `window` object (used to define global variables for a user session). The D3 import is local to the scope of *this* Deno environment, and not the browser. To use it in the browser's console, we'll have to import it within some scope (i.e. inside *some* function). We can also *explicitly* attach it to the global scope (e.g., `window.d3 = d3`) to ensure we're using the same instance of D3 everywhere while debugging, like so: 

```js
function d3Module(require) {
    window.d3 = require('d3');
    const currentListener = window.d3.select("#state-path").on("mouseover");
    console.log(currentListener);
};
```

If we're able to verify that `currentListener` is the callback function inside the Deno-Jupyter notebook (or whatever source we're working from), but *not* inside the browser itself, then the event listener is being overwritten at *some* point. The solution is to understand the DOM lifecycle of our project, as mentioned above. For those hard to catch bugs we can put a `debug` statement inside our code or a breakpoint on any line in any file within the **Source** tab of **Chrome Dev Tools** before a hard refresh (<kbd class="inline">Shift</kbd> + reload if using Chrome on a Mac). 
:::

##### Hooking into The DOM Lifecycle by Using External Scripts

Since this digital garden uses [Quarto](https://github.com/quarto-dev) to generate and publish static pages from Jupyter notes (`.ipynb`)written and executed within a Deno environment, the page that *ultimately* gets rendered inside the browser won't necessarily be the first render. So, a previously attached event listener may have gotten lost in the re-render cycle. Interestingly, the `attr`-based approach doesn't have this problem. Attribute-based event listeners, being part of the markup themselves, get copied over in the re-render process -- but *that* approach can't be used with callback functions. 

In such cases the solution is to *properly* integrate the D3 code with the DOM lifecycle of the framework being relied on. In Quarto's case, we can include the D3 code as a separate `.js` or `.ts` script using the following front matter:

```yaml
include-after-body:
  text: |
    <script type="module" src="./scripts/show-tooltip.js"></script>
```

This script will run at the very end of our page. So we can put the parts of our D3 code that are responsible for the selection of the county `<path>`-s and the attachment of the event listener (for `mouseover`) inside this script. That will ensure the final page retains those event listeners.

Note that if there's shared global state between the external scripts, we must make sure to include the script that contains the global state at the top of the `include-after-body` block (assuming global state is defined in a single script, as is recommended).

What if we aren't using Quarto and, instead, were using a React-like framework? Then we would need to properly integrate our D3 code with *that* framework’s DOM lifecycle instead -- e.g. by using a `componentDidMount` hook in React.

Note that, as in React, we're going to need a way to pass information between components (in this case the map that's defined in the Deno environment and the tooltip defined in the external script). This is similar to how we pass `state` as `props` to a downstream component in React. In this case, however, we're going to use HTML5's custom attributes to pass along the state name as `state-name`, and the `centroid` of a given state as coordinate attributes `cx`, and `cy` to the external script from the Deno environment in which they're defined. 

##### Zoom and Drag

Much like with the tooltips, we can define an external script for zooming into the map on a `zoom` event (e.g. using the mouse wheel or panning).

Here's the zoom-in script. Expand to view.

<details><summary>Click to expand zoom-and-drag code</summary>

```js
import * as d3 from "https://cdn.jsdelivr.net/npm/d3@7/+esm";

// Selects the <svg> zoomable-state-map-wrapper element by `id` and attaches the `reset` callback to its click event
const svg = d3.select("#zoomable-state-map-wrapper")

// Selects the group of states within the <svg>
const groupOfStates = d3.select("#zoomable-group-of-states")

// Tooltip
const tooltip = d3.select("#zoomable-state-name-tooltip");

// Define a zoom behavior in D3 (rrestricting its scaling between 1x-8x, and adding a callback function for zoom events like `pinch`, `mousewheeel`)
const zoom = d3.zoom().scaleExtent([1, 8]).on("zoom", zoomIn);
svg.call(zoom); // Binds the zoom behavior to the map

function zoomIn(event, d) {
    const {transform} = event; // How far the user has zoomed
    groupOfStates
        .attr("transform", transform)
        .attr("stroke-width", 1 / transform.k);
    tooltip
        .style("opacity", "0");
}
```
</details>

Here's the updated fully-interactive map code. Not much has changed, but feel free to expand and examine.

<details><summary>Click to expand interactive map code</summary>

In [ ]:
// Imports: D3, TopoJSON, and Linkedom
import * as d3 from "npm:d3";
import * as topojson from "npm:topojson";
import { DOMParser } from "npm:linkedom";

// Create a fake DOM via Linkedom
const document = new DOMParser().parseFromString(
  `<!DOCTYPE html><html lang="en"><head></head><body></body></html>`,
  "text/html",
);

// Fetch and parse the US map TopoJSON
const url = "https://cdn.jsdelivr.net/npm/us-atlas@3/states-10m.json";
const us = await (await fetch(url)).json();
const states = topojson.feature(us, us.objects.states) as any; // GeoJSON 'FeatureCollection'

// Create an <svg> wrapper in Linkedom's fake DOM using D3
const width = 975;
const height = 610;
const body = d3.select(document.body);
body
  .append("p")
  .text("💡 Tip: Try zoomin in, dragging, or clicking on a state! Click again (or click outside) to zoom out.")
  .style("text-align", "center")
const svg = body
  .append("svg")
    // Add an `id` (the external hook responsible for displaying the tooltip attaches a tooltip element to this wrapper by `id`)
    .attr("id", "zoomable-state-map-wrapper")
    .attr("width", width)
    .attr("height", height)
    .attr("viewBox", `0 0 ${width} ${height}`);
const group = svg.append("g");
const groupOfStates = group.append("g")
    // Add an `id` (the external hook responsible for zoom behavior will select this element by `id`)
    .attr("id", "zoomable-group-of-states")
    .attr("fill", "#EBE8E7")
    .attr("stroke", "#191516")

// Create a geo-projection & a path generator
const projection = d3.geoAlbersUsa().fitSize([width, height], states);
const path = d3.geoPath(projection);


// Append one <path> per state
groupOfStates.selectAll("path")
  .data(states.features)
  .join("path")
    // Define the 'd' attribute of the <path> which gives it its shape
    .attr("d", path)
    // Basic Interactivity: Hover to change color
    .attr("cursor", "pointer")
    .attr("onmouseover", "this.style.fill = '#F9C22E';")
    .attr("onmouseout", "this.style.fill = '#EBE8E7';")
    // Add an `id` (needed for selecting these elements using D3 in the external script that's responsible for displaying the tooltip)
    .attr("id", "zoomable-state-path")
    // Add a custom attribute `state-name` (needed for passing along the state name to the external script)
    .attr("state-name", (d) => {return d.properties.name})
    // Add custom attribute for the centroid of each state <path> (needed to position the tooltip correctly in the external script)
    .attr("cx", (d) => {
      return path.centroid(d)[0]; // Index 0 corresponds to the x-coordinate of the centroid
    })
    .attr("cy", (d) => {
      return path.centroid(d)[1]; // Index 1 corresponds to the y-coordinate of the centroid
    })
    // Custom attributes for the bounding box of each state, needed for the external zoom script 
    .attr("x0", (d) => {
      return path.bounds(d)[0][0]; // Index 0,0 corresponds to the x0 of the bounding box
    })
    .attr("y0", (d) => {
      return path.bounds(d)[0][1];
    })
    .attr("x1", (d) => {
      return path.bounds(d)[1][0];
    })
    .attr("y1", (d) => {
      return path.bounds(d)[1][1];
    })
    // Boolean state to keep track of whether a state <path> has been activated by clicking (or not)
    .attr("activated", "false")
    ;

</details>

When the user interacts with the DOM, D3 computes an object called `event.transform` describing the current transformation (i.e. translation, scaling, keys pressed, and/or other relevant event parameters). This is passed to the callback function for the event, where we can use it to specify the exact transformation. In other words, this object is what captures *how much* the user has zoomed in using their mouse wheel, or *how far* they've panned. 

**Zoom Behavior - General Strokes**

We define a new zoom behavior called `zoom`. We can assign an event listener to a behavior using `on` (much like when assigning listeners to D3 selections of DOM elements). Later, when we *bind* the behavior to a DOM element, D3 will attach the relevant event listeners to the DOM object itself. Inside the callback function of the event listener, `zoomIn`, we can take the `event.transform` object and apply the same, *exact* transformation to `groupOfStates`. We also need to narrow the stroke width, as we zoom, by the inverse relationship `1/transform.k` where `transform.k` is the scalar magnification of `event.transform` (i.e. if `k` is `1` then the zoom level is 1x)

Note that the`d3.zoom` behavior works with `selection.on` to attach the necessary listeners. It also cannot be *directly* called on a `d3.selection`, it must be *bound* to one using a `selection.call`. We use this to bind the zoom behavior to the `<svg>` map wrapper above. Note, also, that `selection.call(zoom)` is just an alternate syntax for `zoom.transform(selection)` (nothing but a `d3.zoom` behavior applied, by its `transform` method, to a selection).

Taking a step back we can look at the big picture. What we've done is, essentially, instruct the browser to apply a zoom behavior to the `groupOfStates` when its wrapper's (`svg`'s) `zoom` event is triggered. 

##### Zooming into a State

We would like to zoom into a state by clicking into it. We'd also like to reset the zoom whenever a user clicks outside a state, or inside an already clicked state. Here'We can lump in this code with the rest of the zoom script (`zoom-in.js`) above. 

First, the `svg` (i.e. `id = zoomable-state-map-wrapper`), gets a callback function `resetView` for its `click` event. That callback, `resetView`, is invoked whenever a user clicks the empty space (i.e. outside a state, with `id = zoomable-state-path`, but *inside* the `svg` wrapper.

In the last section, we saw how to get the zoom state from `event.transform`. In this section, we will use D3 to construct a zoom state using the D3 *behavior* `zoom`. From the D3 [docs](https://d3js.org/d3-zoom#_zoom):

> The zoom behavior stores the zoom state on the element to which the zoom behavior was applied, not on the zoom behavior itself. This allows the zoom behavior to be applied to many elements simultaneously with independent zooming. The zoom state can change either on user interaction or programmatically via `zoom.transform`.

Method `zoom.transform` is a member of the `zoom` behavior responsible for setting the zoom state of the given selection according to a `transform` (an `event.transform` object) -- like the one we've encountered before. A `transform` of a `zoom` behavior is an object with three main properties:

* `transform.k`: the current scale factor
* `transform.x` and `transform.y`: the current translation offsets in pixels (i.e. the pan).

For example, calling:
```js
svg.call(
  zoom.transform,
  d3.zoomIdentity
    .translate(100, 50)
    .scale(2)
);
```
Tells the zoom behavior to pan the content by `100`, and `50` and zoom by a factor of `2`. The call method takes a function (in this case zoom.transform) which itself takes a transform as an argument (the second argument to the call method). This is roughly equivalent to: 

```js
zoom.transform(svg, d3.zoomIdentity.translate(100,50).scale(2));
```

Note that in the section above, we took the `event.transform` object from the DOM element and used its `k` magnification constant to define the correct zoom behavior. Now, we've seen how to *construct* a `transform` object programmatically by using D3! Transforms are all, ultimately, attached to the elements in the DOM as HTML attributes of the same name (e.g. `<svg ... transform=translate(120,0)> ... </svg>`)

Let's take a look at the `resetView` code.

<details><summary>**Click to expand code to reset the zoom level**</summary>


```js
/*--------------------------------- Global State -----------------------------------*/
const width = svg.attr("width");
const height = svg.attr("height");
/*----------------------------------------------------------------------------------*/

// Reset zoom level
function resetView(event, d) {
    groupOfstates.transition().style("fill", null);
    svg.transition().duration(750).call(
        zoom.transform,
        d3.zoomIdentity,
        d3.zoomTransform(svg.node()).invert([width / 2, height / 2])
    );
}
```
</details>

**Reset Zoom - General Strokes**

First, the function resets the fill on all states (removing any highlighted color). Then it binds a default D3 zoom behavior (`zoom.transform`), providing parameters:

* `d3.zoomIdentity`: the un-zoomed state
* The center of the `svg` in the original coordinate system (before any zoom or pan):
    * `d3.zoomTransform(svg.node())`: gets the current transform (the current scale `k` and translations `x`,`y`) that are in effect
    * `invert([width/2, height/2])`: applies the *inverse* of that transform to the point midpoint of the `svg` (the point on coordinate `[width/2 ,height/2`). This tells us where the midpoint lies in the original coordinate system (before any zoom or pan)

Note that `selection.call` above provides the parameters to `zoom.transform` in-line. These are the same parameters taken by `zoom.transform` were it to be called directly. 

We also add zooming into a state on a click event. Expand the code block below to see how we define the `zoomInto` callback function.

<details><summary>Click to expand code to zoom on click</summary>

```js
// Zoom into state
function zoomInto(event, d) {
    
    // Get bounding box coordinates
    const x0 = parseInt(event.target.getAttribute("x0"));
    const y0 = parseInt(event.target.getAttribute("y0"));
    const x1 = parseInt(event.target.getAttribute("x1"));
    const y1 = parseInt(event.target.getAttribute("y1"));
    // Get global boolean state (clicked/unclicked)
    activated = event.target.getAttribute("activated") == "true" ? true : false;

    // Use for debugging
    // console.log(activated);

    event.stopPropagation();

    if (activated) { // Zoom out if a click event is registered on an activated state
        resetView(event, d); // resetView itself sets attr `activated` to `false`
    }  
    if (!activated) { // Zoom in otherwise, and activate state
        states.transition().style("fill", null);
        svg.transition().duration(750).call(
            zoom.transform,
            d3.zoomIdentity
                .translate(width / 2, height / 2)
                .scale(Math.min(8, 0.9 / Math.max((x1 - x0) / width, (y1 - y0) / height))) // Never exceed max scale
                .translate(-(x0 + x1) / 2, - (y0 + y1) / 2),
            d3.pointer(event, svg.node())
        );
        d3.selectAll("#zoomable-state-path").attr("activated", "true");
    }
}
```
</details>

The line `event.stopPropagation()` is used to ensure that the click event on a state `<path>` does not trigger any other click event listeners on parent elements (such as the `<svg>` wrapper). We could've also used it in other callback functions (as an abundance of caution).

The final argument `d3.pointer(event, svg.node())` just ensures that the zoom transition is anchored around the pointer position (so it looks more natural as we click).

Additionally, we want to hide the tooltips as soon as the user has interacted with the map *in any way* (either by dragging, clicking into a state, or zooming).

Here's the final external script `zoom-in.js` which contains all the drag and zoom behavior in one file.

<details><summary>Click to expand full zoom-and-drag code</summary>

```js
import * as d3 from "https://cdn.jsdelivr.net/npm/d3@7/+esm";

/*---------------------------------------*/
////////////////// MAP 2 //////////////////
/*---------------------------------------*/

// Selects the <svg> map wrapper element by `id` and attaches the `reset` callback to its click event
const svg = d3.select("#zoomable-state-map-wrapper")
    .on("click", resetView);

// Tooltip
const tooltip = d3.select("#zoomable-state-name-tooltip");

// Selects the group of states within the <svg>
const groupOfStates = d3.select("#zoomable-group-of-states");

/*------------------------------------------*/
////////////////// Behavior //////////////////
/*------------------------------------------*/

/*--------------------------------- Global State -----------------------------------*/
let activated = false; // Keeps track of whether a state has been clicked into or not
// Get properties of <svg> wrapper...
const width = svg.attr("width");
const height = svg.attr("height");
/*----------------------------------------------------------------------------------*/

// Define a zoom behavior in D3
// Restrict scaling between 1x-8x 
// Add a callback function for zoom events like drag, and mousewheeel
const zoom = d3.zoom()
    .scaleExtent([1, 8])
    .on("zoom", zoomIn);
// Binds the zoom behavior to the <svg> (`#zoomable-map-container`)
svg.call(zoom);

// Zooms in using mousewheel, pans using drag
function zoomIn(event, d) {
    const {transform} = event;
    groupOfStates
        .attr("transform", transform)
        .attr("stroke-width", 1 / transform.k);
    tooltip
        .style("opacity", "0");
}    

// Selects the states (elements with `id=zoomable-state-path`) and attaches an event-listener for a `click` event
const states = d3.selectAll("#zoomable-state-path")
    .on("click", zoomInto)

// Reset zoom level
function resetView(event, d) {
    groupOfStates.transition().style("fill", null);
    svg.transition().duration(750).call(
        zoom.transform,
        d3.zoomIdentity,
        d3.zoomTransform(svg.node()).invert([width / 2, height / 2])
    );
    tooltip
        .style("opacity", "0");
    d3.selectAll("#zoomable-state-path").attr("activated", "false");
}

// Zoom into state
function zoomInto(event, d) {
    
    // Get bounding box coordinates
    const x0 = parseInt(event.target.getAttribute("x0"));
    const y0 = parseInt(event.target.getAttribute("y0"));
    const x1 = parseInt(event.target.getAttribute("x1"));
    const y1 = parseInt(event.target.getAttribute("y1"));
    // Get global boolean state (clicked/unclicked)
    activated = event.target.getAttribute("activated") == "true" ? true : false;

    // Use for debugging
    // console.log(activated);

    event.stopPropagation();

    if (activated) { // Zoom out if a click event is registered on an activated state
        resetView(event, d); // resetView itself sets attr `activated` to `false`
    }  
    if (!activated) { // Zoom in otherwise, and activate state
        states.transition().style("fill", null);
        svg.transition().duration(750).call(
            zoom.transform,
            d3.zoomIdentity
                .translate(width / 2, height / 2)
                .scale(Math.min(8, 0.9 / Math.max((x1 - x0) / width, (y1 - y0) / height))) // Never exceed max scale
                .translate(-(x0 + x1) / 2, - (y0 + y1) / 2),
            d3.pointer(event, svg.node())
        );
        d3.selectAll("#zoomable-state-path").attr("activated", "true");
    }
}
```
</details>

Let's see zoom and drag in action!

In [30]:
// Get the new markup from Linkedom to display it
const htmlOutput = document.documentElement.outerHTML; // Entire <html>…</html>

// Use 'Deno.jupyter.display' to display the raw <html> string, now containing the county <paths>-s, as actual HTML
await Deno.jupyter.display({
  "text/html": htmlOutput,
}, { raw: true });

💡 Tip: Try zoomin in, dragging, or clicking on a state! Click again (or click outside) to zoom out. <path activated="false" y1="480.71434197353415" x1="738.3236739367742" y0="372.1672157529463" x0="671.8325500230226" cy="422.637486026742" cx="700.7386861292639" state-name="Alabama" id="zoomable-state-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M677.986,480.51L681.705,479.622L681.734,479.149L682.737,479.819L682.176,480.392L681.478,480.149L680.27,480.286L678.951,480.714ZM671.856,376.314L671.833,376.032L681.341,375.336L685.329,374.981L695.477,374.28L701.363,373.74L703.7,373.467L713.76,372.492L717.625,372.167L719.238,378.089L721.621,386.24L723.563,393.221L725.015,398.64L726.636,404.36L730.288,417.473L730.94,418.005L730.758,418.709L731.699,419.454L732.345,422.206L733.151,423.478L734.408,424.641L734.782,426.152L735.448,426.757L735.05,429.021L736.287,429.408L737.159,430.064L736.619,431.113L735.899,431.197L736.083,431.63L735.212,432.13L734.474,433.201L734.791,434.225L734.846,435.848L734.573,437.509L733.808,438.96L733.707,439.824L734.368,442.096L734.331,442.934L735.77,444.415L736.228,446.634L735.889,447.752L736.114,448.54L735.757,450.095L735.993,451.562L735.575,452.008L736.016,452.922L735.933,453.953L737.456,455.498L738.059,456.639L738.324,457.762L733.873,458.274L729.165,458.897L721.86,459.767L710.286,460.964L701.288,461.71L690.237,462.762L690.5,463.455L689.823,465.683L689.995,466.102L691.662,467.443L691.93,468.192L694.471,469.472L694.789,470.744L693.957,472.741L694.251,473.751L695.679,474.583L694.531,475.346L694.132,476.927L693.553,477.262L694.063,477.668L693.099,478.244L690.603,479.125L687.953,479.808L684.192,480.205L685.338,479.336L686.186,479.753L688.496,479.045L688.643,478.393L687.557,477.583L687.018,476.819L685.6,476.056L684.987,474.413L685.472,472.978L685.15,471.573L684.605,470.716L683.216,470.23L682.285,471.175L682.478,471.875L681.964,472.991L682.095,473.753L681.743,474.385L681.9,477.083L681.465,478.362L680.463,478.52L680.475,477.841L679.071,477.204L678.087,477.585L677.717,477.182L676.541,477.691L676.007,473.554L675.272,466.888L674.239,458.998L672.425,444.56L672.595,439.204L672.903,425.766L672.908,421.758L673.33,400.534L673.468,397.494L673.684,385.297L673.881,378.421L672.812,377.846Z" /><path activated="false" y1="587.8605689458193" x1="250.35186397980095" y0="453.74685748545426" x0="-1.1368683772161603e-13" cy="510.1736115713112" cx="153.70158104833692" state-name="Alaska" id="zoomable-state-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M162.053,531.019L162.118,530.765L162.358,530.823L162.112,531.121ZM160.778,544.929L161.151,544.318L161.365,544.419L161.366,544.722L161.002,545.228L160.787,545.113ZM157.91,546.468L158.277,546.546L158.244,546.784ZM157.173,546.613L157.485,546.714L157.394,546.934L157.195,546.849ZM156.605,546.375L156.885,546.379L156.823,546.605ZM156.723,553.623L156.829,553.323L157.119,553.4L157.02,553.883L156.807,554.002ZM155.834,548.248L155.997,548.059L156.301,548.313L155.836,548.34ZM155.477,537.365L155.853,536.834L155.938,536.094L156.36,536.271L155.997,536.704L155.886,537.03L155.958,537.186ZM154.723,548.35L154.911,547.983L155.169,548.161L155.494,548.178L155.035,548.408ZM153.414,538.804L153.644,538.76L153.724,539.338ZM150.61,554.937L151.022,554.348L151.238,554.306L151.618,554.736L151.689,554.603L151.381,554.18L151.463,553.824L151.592,553.756L151.982,553.915L152.282,553.786L151.861,553.437L152.085,553.066L152.504,553.263L152.719,553.233L152.512,552.868L152.682,552.754L152.971,552.88L153.166,552.581L152.908,552.546L152.718,552.332L153.004,552.308L153.31,551.861L153.793,551.733L153.617,551.507L153.565,551.06L153.94,550.618L154.044,550.82L154.79,550.467L154.89,550.511L154.739,551.291L154.556,551.413L154.201,552.099L154.285,552.65L154.799,552.565L154.816,552.111L155.132,552.162L155.445,552.621L155.722,552.298L155.9,552.51L155.678,553.286L155.812,553.427L155.927,552.95L156.305,552.629L156.41,

### Adding Counties

Using the same tactics developed so far, let's import and display the counties. 

<details><summary>Click to expand updated interactive map code</summary>

In [ ]:
// Imports: D3, TopoJSON, and Linkedom
import * as d3 from "npm:d3";
import * as topojson from "npm:topojson";
import { DOMParser } from "npm:linkedom";

// Create a fake DOM via Linkedom
const document = new DOMParser().parseFromString(
  `<!DOCTYPE html><html lang="en"><head></head><body></body></html>`,
  "text/html",
);

// Fetch and parse the US map TopoJSON
const statesUrl = "https://cdn.jsdelivr.net/npm/us-atlas@3/states-10m.json";
const usStates = await (await fetch(statesUrl)).json();
const states = topojson.feature(usStates, usStates.objects.states) as any; // GeoJSON 'FeatureCollection'
const countiesUrl = "https://cdn.jsdelivr.net/npm/us-atlas@3/counties-10m.json";
const usCounties = await (await fetch(countiesUrl)).json();
const counties = topojson.feature(usCounties, usCounties.objects.counties) as any; // GeoJSON 'FeatureCollection'


// Create an <svg> wrapper in Linkedom's fake DOM using D3
const width = 975;
const height = 610;
const body = d3.select(document.body);
body
  .append("p")
  .text("💡 Tip: Try zoomin in, dragging, or clicking on a state! Click again (or click outside) to zoom out.")
  .style("text-align", "center")
const svg = body
  .append("svg")
    // Add an `id` (the external hook responsible for displaying the tooltip attaches a tooltip element to this wrapper by `id`)
    .attr("id", "zoomable-county-map-wrapper")
    .attr("width", width)
    .attr("height", height)
    .attr("viewBox", `0 0 ${width} ${height}`);
const group = svg.append("g");
const groupOfCounties = group.append("g")
    // Add an `id` (the external hook responsible for zoom behavior will select this element by `id`)
    .attr("id", "zoomable-group-of-counties")
    .attr("fill", "#EBE8E7")
    .attr("stroke", "#191516")

// Create a geo-projection & a path generator
const projection = d3.geoAlbersUsa().fitSize([width, height], states);
const path = d3.geoPath(projection);


// Append one <path> per county
groupOfCounties.selectAll("path")
  .data(counties.features)
  .join("path")
    // Define the 'd' attribute of the <path> which gives it its shape
    .attr("d", path)
    // Basic Interactivity: Hover to change color
    .attr("cursor", "pointer")
    .attr("onmouseover", "this.style.fill = '#F9C22E';")
    .attr("onmouseout", "this.style.fill = '#EBE8E7';")
    // Add an `id` (needed for selecting these elements using D3 in the external script that's responsible for displaying the tooltip)
    .attr("id", "zoomable-county-path")
    // Add a custom attribute `county-name` (needed for passing along the county name to the external script)
    .attr("county-name", (d) => {return d.properties.name})
    // Add custom attribute for the centroid of each county <path> (needed to position the tooltip correctly in the external script)
    .attr("cx", (d) => {
      return path.centroid(d)[0]; // Index 0 corresponds to the x-coordinate of the centroid
    })
    .attr("cy", (d) => {
      return path.centroid(d)[1]; // Index 1 corresponds to the y-coordinate of the centroid
    })
    // Custom attributes for the bounding box of each county, needed for the external zoom script 
    .attr("x0", (d) => {
      return path.bounds(d)[0][0]; // Index 0,0 corresponds to the x0 of the bounding box
    })
    .attr("y0", (d) => {
      return path.bounds(d)[0][1];
    })
    .attr("x1", (d) => {
      return path.bounds(d)[1][0];
    })
    .attr("y1", (d) => {
      return path.bounds(d)[1][1];
    })
    // Boolean county to keep track of whether a county <path> has been activated by clicking (or not)
    .attr("activated", "false")
    ;

</details>

Let's generate the map one more time!

In [38]:
// Get the new markup from Linkedom to display it
const htmlOutput = document.documentElement.outerHTML; // Entire <html>…</html>

// Use 'Deno.jupyter.display' to display the raw <html> string, now containing the county <paths>-s, as actual HTML
await Deno.jupyter.display({
  "text/html": htmlOutput,
}, { raw: true });

💡 Tip: Try zoomin in, dragging, or clicking on a state! Click again (or click outside) to zoom out. <path activated="false" y1="369.90525617714104" x1="250.18655576858333" y0="308.38717143675" x0="205.4869688463396" cy="337.4714538652149" cx="224.07913894119278" county-name="Mohave" id="zoomable-county-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M223.845,311.785L224.499,308.387L236.664,310.678L244.032,312.02L250.187,313.114L250.125,314.178L249.164,315.353L248.201,317.238L248.201,317.238L247.806,318.762L247.806,318.762L247.228,319.889L247.255,321.525L246.474,322.366L246.309,323.297L246.746,324.047L246.286,324.503L246.45,325.722L245.563,326.794L244.563,326.713L243.026,327.86L241.789,327.677L241.064,328.181L239.623,328.42L238.758,328.283L236.72,329.354L236.381,329.119L235.55,329.486L235.431,330.137L234.754,330.705L233.498,330.262L232.563,331.186L232.487,332.633L232.827,333.214L232.323,334.217L232.638,334.928L231.952,336.224L230.787,342.543L225.96,368.714L225.363,369.122L222.395,368.244L221.245,369.508L219.472,369.905L218.522,369.054L216.693,369.053L216.609,368.461L215.278,368.246L213.43,367.661L212.649,366.627L211.785,366.364L211.286,365.229L210.474,364.707L208.886,362.474L208.095,362.184L208.431,361.134L207.67,356.386L206.462,354.935L206.277,353.57L205.487,352.316L206.09,349.605L206.813,347.919L206.272,347.388L207.737,346.85L208.001,345.516L208.096,342.106L207.962,340.828L207.454,339.821L207.42,338.697L207.924,338.104L208.304,336.195L207.888,335.38L208.217,334.694L207.899,334.033L208.273,333.125L208.182,331.97L208.636,330.981L209.279,330.646L208.777,329.949L208.397,328.004L208.803,327.256L208.669,325.687L210.319,325.418L210.979,324.991L211.234,325.311L212.063,324.994L213.103,325.219L213.135,325.706L214.679,325.644L215.466,325.856L216.315,327.39L216.091,327.885L217.199,329.083L218.716,329.165L220.848,326.053L221.208,325.834L222.09,321.576L222.939,316.502Z" /><path activated="false" y1="482.9731826103117" x1="643.6491155459507" y0="466.4887167041835" x0="635.0792937476679" cy="474.7124866700605" cx="638.579164180424" county-name="Tangipahoa" id="zoomable-county-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M635.079,466.726L635.413,466.707L639.151,466.489L639.274,468.551L639.895,468.834L641.063,471.511L641.267,472.665L641.554,474.077L641.931,480.296L643.008,481.34L643.649,482.973L640.943,481.822L637.532,482.418L637.364,480.699L636.685,479.46L636.194,479.131L635.72,477.918L635.516,474.357Z" /> <path activated="false" y1="107.81443576555694" x1="540.5493436551791" y0="93.12374966981429" x0="517.1021999027435" cy="101.84079299992663" cx="528.010659028198" county-name="Polk" id="zoomable-county-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M517.102,93.124L526.58,93.187L526.563,96.492L526.826,96.493L526.82,97.738L526.807,100.246L528.712,100.273L528.705,102.151L536.336,102.14L536.333,100.262L538.185,100.239L538.18,98.363L540.082,98.393L540.16,104.024L540.532,104.023L540.549,107.781L532.941,107.814L521.289,107.758L521.257,105.339L520.992,105.281L520.739,104.014L520.374,103.341L520.07,101.906L519.649,101.586L519.086,99.703L518.939,98.623L518.365,97.985L518.126,95.735L517.709,95.322Z" /><path activated="false" y1="126.60721313765953" x1="522.5208728451287" y0="113.20946533641177" x0="508.4243591561242" cy="119.9691703590834" cx="515.2541674504915" county-name="Cass" id="zoomable-county-path" onmouseout="this.style.fill = '#EBE8E7';" onmouseover="this.style.fill = '#F9C22E';" cursor="pointer" d="M508.526,113.209L512.318,113.291L521.505,113.42L521.542,115.321L521.696,116.068L521.615,119.254L522.197,120.21L522.521,120.008L522.181,122.187L521.909,122.502L522.215,124.594L521.935,125.861L522.037,126.552L521.929,126.607L514.704,126.547L508.666,126.453L508.8,118.88L508.424,118.874Z" /> <path activated="false" y1="181.12371628626227" x1="878.9903177233039"

## Conclusion

In this post, we explored how to set up and use D3.js within a Jupyter notebook using Deno. We started by setting up the environment and verifying that we could run JavaScript code inside the notebook. We then fetched and examined geographic data in TopoJSON format and used D3 to render an interactive map of the United States. We added interactivity features such as hover effects, tooltips, zoom, and drag behaviors to enhance the user experience. Finally, we extended the map to include county-level granularity.

We can take this project into multiple directions, depending on the kind of data we'd like to visualize. Stay tuned for an extension of this post into useful heatmaps (e.g. crime, average salary, average age of population, etc.). 